# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании. Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Оглавление<a name="Оглавление"></a>

- 1. [Загрузка данных](#Загрузка)
  - 1.1. [Вывод](#Вывод)
- 2. [Умножение матриц](#Умножение)
- 3. [Алгоритм преобразования](#Алгоритм)
- 4. [Проверка алгоритма](#Проверка)
  - 4.1. [Обучим и сравним показатель r2_score](#Обучим)
- 5. [Вывод](#Вывод_)

## 1. Загрузка данных<a name="Загрузка"></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_spd_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('./datasets/insurance.csv')

In [3]:
df.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Преобразуем тип в инт, так как возраст в целых числах измеряется.

In [7]:
df['Возраст'] = df['Возраст'].astype('int')

[К оглавлению](#Оглавление)

### 1.1. Вывод<a name="Вывод"></a>

Данные загружены и не имеют аномалий. Приведение типов позволит более быстро обращаться к данным.

## 2. Умножение матриц<a name="Умножение"></a>

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится.

**Обоснование:** 

Предсказания для новой матрицы:
$$
ap = XPw
$$

Вектор весов:
$$
w = ((XP)^T(XP))^{-1}(XP)^Ty
$$

Подставим в формулу расчёта предсказаний значение вектора весов:
$$
ap = (XP)((XP)^T(XP))^{-1}(XP)^Ty
$$

Раскроем скобки с транспонированием:
$$
ap = (XP)(P^TX^TXP)^{-1}P^TX^Ty
$$

Выносим *P* за скобки, так как она квадратная (по условию):
$$
ap = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

Ввиду того, что выражения:
$$
PP^{-1}
$$

и
$$
P^T (P^T)^{-1}
$$

равны *E*, получаем:
$$
ap = X(X^T X)^{-1} X^T y
$$

Из чего следует:
$$
ap = Xw = a
$$

[К оглавлению](#Оглавление)

## 3. Алгоритм преобразования<a name="Алгоритм"></a>

**Алгоритм**

Запись в переменные признаков и целевого признака

Создание рандомной обратной матрицы

Обучение модели без преобразования

Получение метрики R2 на модели без преобразования

Умножение обратной матрицы на матрицу признаков

Умножение признаков на обратимую матрицу

Обучение модели на преобразованых данных

Получение мерики R2 на преобразованых данных

Сравнение метрик

**Обоснование**

Мы умножим наши признаки на квадратную невырожденную матрицу и сравним качество линейной регресии на двух матрицах: умноженной на квадратную невырожденную матрицу и не умноженой на неё.

[К оглавлению](#Оглавление)

## 4. Проверка алгоритма<a name="Проверка"></a>

In [8]:
X = df.drop(['Страховые выплаты'], axis=1)
y = df['Страховые выплаты']

In [9]:
X_sample = X.shape[1]
det = 0
while det == 0:
    r = np.random.randint(100)
    matrix_x = make_spd_matrix(n_dim = X_sample, random_state = r)
    det = np.linalg.det(matrix_x)

In [10]:
X_new = X.values @ matrix_x
X_new

array([[ 67589.86983569,  25620.15937109,  54377.98235696,
        -30693.77520317],
       [ 51789.60774837,  19640.41519763,  41666.88781771,
        -23519.26895626],
       [ 28623.62910663,  10857.21510874,  23028.62589239,
        -12999.59309383],
       ...,
       [ 46186.3644544 ,  17502.66683202,  37159.71359665,
        -20972.56083894],
       [ 44555.18984374,  16885.74653422,  35846.42207685,
        -20230.68644503],
       [ 55322.21726425,  20966.57117387,  44508.27651076,
        -25122.31951978]])

[К оглавлению](#Оглавление)

### 4.1. Обучим и сравним показатель r2_score<a name="Обучим"></a>

In [11]:
linreg = LinearRegression()

linreg.fit(X, y)
predict1 = linreg.predict(X)

linreg.fit(X_new, y)
predict2 = linreg.predict(X_new)

print(np.isclose(r2_score(y, predict1), r2_score(y, predict2)))
print(r2_score(y, predict1), r2_score(y, predict2))

True
0.4249455028666801 0.42494550286667365


[К оглавлению](#Оглавление)

### 5. Вывод<a name="Вывод_"></a>

Качество линейной регрессии не изменилось, метрика R2 сильно не меняет совего значения, значит можно сделать вывод,что алгоритм преобразования данных работает правильно.